--> To put content in collapsable format, we can use the following snippet in the markdowns:

<details>
<summary style="cursor: pointer">
<b> double click the markdown to see the code instead of this </b>
</summary>

# 3.3 Dimentionality Reduction-based Techniques

-- Feature projections and transformation-based importance

- Principal Component Analysis (PCA)
- t-SNE (for visualization, not selection)
- UMAP
- Autoencoder Feature Compression
- Factor Analysis

-----------

## 3.3.1 Principal Component Analysis (PCA)

<details>
<summary style="cursor: pointer">
<h2> { Understanding Principal Component Analysis (PCA) } </h2>
</summary>
<h3> What is PCA? </h3>
<p> PCA is a statistical technique that transforms high-dimensional data into a lower-dimensional space by finding new axes (principal components) that capture the most variance in the data.</p>
<h3> It's role in Feature Importance / Compression: </h3>
<ul>
    <li> PCA does not use the target variable—it's unsupervised.</li>
    <li> Reduces dimensionality while retaining as much variance as possible.</li>
    <li> Helps in noise reduction, visualization, and speeding up ML algorithms.</li>
</ul>
<h3> Resources: </h3>
<ol>
    <li><a href="https://www.youtube.com/watch?v=FgakZw6K1QQ" target="_blank">PCA Explained Visually (StatQuest)</a></li>
</ol>
</details>

##### Parameters:

- X: Features (DataFrame)
- model: A fitted model (optional, to align PCA-transformed features with model-specific importance)
- n_components: Number of principal components to compute (default: keep all)
- scale_data: Whether to standardize features before PCA (important for PCA accuracy)
- plot_size: Tuple indicating the size of the plot (width, height)

##### Returns:

- DataFrame containing features and their overall absolute contribution scores (based on PCA loadings)
- Displays a styled bar plot (if show_plot=True)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pca_feature_importance(X,
                            model=None,
                            n_components=None,
                            scale_data=True,
                            top_n_features=20,
                            show_plot=True,
                            plot_size=(12, 8)):

    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    feature_names = X.columns.tolist()
    X_processed = X.copy()

    if scale_data:
        scaler = StandardScaler()
        X_processed = scaler.fit_transform(X_processed)
    else:
        X_processed = X_processed.values

    pca = PCA(n_components=n_components)
    pca.fit(X_processed)

    # Feature importance based on PCA loadings
    loadings = pca.components_.T

    # Sum of absolute loadings across all principal components
    feature_contributions = np.sum(np.abs(loadings), axis=1)

    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'PCA_Contribution': feature_contributions
    }).sort_values('PCA_Contribution', ascending=False)

    # Optionally add model feature importances if provided
    if model is not None and hasattr(model, 'feature_importances_'):
        importance_df['Model_Importance'] = model.feature_importances_
    elif model is not None and hasattr(model, 'coef_'):
        importance_df['Model_Importance'] = np.abs(model.coef_).flatten()

    if top_n_features:
        importance_df = importance_df.head(top_n_features)

    if show_plot:
        plt.figure(figsize=plot_size)
        importance_df_sorted = importance_df.sort_values('PCA_Contribution', ascending=True)

        colors = plt.cm.cool(np.linspace(0.2, 1, len(importance_df_sorted)))
        bars = plt.barh(importance_df_sorted['Feature'],
                        importance_df_sorted['PCA_Contribution'],
                        color=colors,
                        alpha=0.9)

        plt.xlabel('Overall Absolute Contribution (PCA)', fontsize=12)
        plt.title('PCA Feature Importance (via Loadings)', fontsize=14, pad=20)
        plt.grid(axis='x', linestyle='--', alpha=0.3)

        for bar in bars:
            width = bar.get_width()
            plt.text(width + 0.005,
                     bar.get_y() + bar.get_height() / 2,
                     f'{width:.4f}',
                     va='center',
                     fontsize=9)

        method_text = (
            f"Method: PCA Loadings\n"
            f"Scale Data: {scale_data}\n"
            f"Model-Specific: {'Yes' if model else 'No'}"
        )
        plt.annotate(method_text,
                     xy=(0.98, 0.02),
                     xycoords='axes fraction',
                     ha='right',
                     va='bottom',
                     fontsize=9,
                     bbox=dict(boxstyle='round', alpha=0.1))

        plt.tight_layout()
        plt.show()

    return importance_df

------------

## 3.3.2 t-SNE (for visualization, not selection)

<details>
<summary style="cursor: pointer">
<h2> { Understanding t-SNE for Visualization } </h2>
</summary>
<h3> What is t-SNE? </h3>
<p> t-SNE is a non-linear dimensionality reduction technique that projects high-dimensional data into 2D or 3D for visualization while preserving local neighborhood structure.</p>
<h3> It's role in Feature Importance / Visualization: </h3>
<ul>
    <li> Primarily used for exploring and visualizing data—not for feature selection or compression.</li>
    <li> Useful for understanding clusters and patterns in the dataset.</li>
    <li> Not ideal for training downstream ML models due to its stochastic nature.</li>
</ul>
<h3> Resources: </h3>
<ol>
    <li><a href="https://www.youtube.com/watch?v=NEaUSP4YerM" target="_blank">t-SNE Intuition and Application</a></li>
</ol>
</details>

##### Parameters:
    
- X: Features (DataFrame)
- y: Target (Series or 1D array)
- model: A fitted model (optional, to color points by predicted classes)
- perplexity: t-SNE perplexity parameter (related to number of nearest neighbors)
- n_iter: Number of optimization iterations
- learning_rate: Learning rate for t-SNE updates
- scale_data: Whether to standardize features before t-SNE
- figsize: Tuple indicating the size of the plot (width, height)

##### Returns:
    
- DataFrame containing 2D t-SNE coordinates with true labels (and optionally predicted labels if model is given)
- Displays a 2D scatter plot colored by class

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

def tsne_feature_visualization(X,
                                y,
                                model=None,
                                perplexity=30,
                                n_iter=1000,
                                learning_rate=200,
                                scale_data=True,
                                random_state=42,
                                figsize=(12, 8)):

    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    X_processed = X.copy()

    if scale_data:
        scaler = StandardScaler()
        X_processed = scaler.fit_transform(X_processed)
    else:
        X_processed = X_processed.values

    tsne = TSNE(n_components=2,
                perplexity=perplexity,
                n_iter=n_iter,
                learning_rate=learning_rate,
                random_state=random_state)

    X_embedded = tsne.fit_transform(X_processed)

    tsne_df = pd.DataFrame({
        'TSNE_1': X_embedded[:, 0],
        'TSNE_2': X_embedded[:, 1],
        'True_Label': y
    })

    if model is not None:
        preds = model.predict(X)
        tsne_df['Predicted_Label'] = preds

    # Plotting
    plt.figure(figsize=figsize)
    
    if model is None:
        groups = tsne_df.groupby('True_Label')
        title = 't-SNE Visualization by True Labels'
    else:
        groups = tsne_df.groupby('Predicted_Label')
        title = 't-SNE Visualization by Model Predictions'

    colors = plt.cm.Set1(np.linspace(0, 1, len(groups)))

    for (label, group), color in zip(groups, colors):
        plt.scatter(group['TSNE_1'],
                    group['TSNE_2'],
                    label=label,
                    alpha=0.7,
                    s=60,
                    edgecolors='k',
                    linewidth=0.5,
                    color=color)

    plt.title(title, fontsize=16)
    plt.xlabel('TSNE 1', fontsize=12)
    plt.ylabel('TSNE 2', fontsize=12)
    plt.legend(title='Class', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()

    return tsne_df

-------------

## 3.3.3 UMAP

<details>
<summary style="cursor: pointer">
<h2> { Understanding UMAP for Dimensionality Reduction } </h2>
</summary>
<h3> What is UMAP? </h3>
<p> UMAP is a non-linear dimensionality reduction technique that preserves both global and local data structures. It's faster and often more effective than t-SNE for many tasks.</p>
<h3> It's role in Feature Importance / Visualization: </h3>
<ul>
    <li> Suitable for both visualization and downstream ML applications.</li>
    <li> Retains more of the global structure compared to t-SNE.</li>
    <li> Can be used for clustering, anomaly detection, and data exploration.</li>
</ul>
<h3> Resources: </h3>
<ol>
    <li><a href="https://umap-learn.readthedocs.io/en/latest/" target="_blank">UMAP Official Documentation</a></li>
</ol>
</details>

##### Parameters:

- X: Features (DataFrame)
- model: A fitted model (optional, for aligning compressed features with model performance)
- n_components: Number of compressed dimensions (default=2)
- n_neighbors: Size of local neighborhood (default=15)
- min_dist: Minimum distance between points in low-dimensional space (default=0.1)
- metric: Distance metric for UMAP (default='euclidean')
- scale_data: Whether to standardize features before compression
- plot_size: Tuple indicating plot size (width, height)

##### Returns:

- UMAP transformer (to transform features)
- DataFrame of compressed feature representations
- Displays a plot of feature contributions (if show_plot=True)

In [8]:
!pip install umap

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3550 sha256=836c652fd923d66d8392472788f8169536a6de5c19955190bc46bab3e9bc49a1
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\6e\d3\96\de108ab40279ac7d1ba174a5c40b25dc758a6b3d371617cdea
Successfully built umap


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

def umap_feature_compression(X,
                              model=None,
                              n_components=2,
                              n_neighbors=15,
                              min_dist=0.1,
                              metric='euclidean',
                              random_state=42,
                              scale_data=True,
                              show_plot=True,
                              plot_size=(12, 8)):

    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    np.random.seed(random_state)

    feature_names = X.columns.tolist()
    X_processed = X.copy()

    if scale_data:
        scaler = StandardScaler()
        X_processed = scaler.fit_transform(X_processed)
    else:
        X_processed = X_processed.values

    # Build and Fit UMAP
    reducer = umap.UMAP(
        n_components=n_components,
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric=metric,
        random_state=random_state
    )
    embedding = reducer.fit_transform(X_processed)
    compressed_df = pd.DataFrame(embedding, columns=[f'UMAP_{i+1}' for i in range(n_components)])

    if show_plot:
        # Feature Importance Approximation: Permute each feature and measure embedding distortion
        baseline_embedding = reducer.transform(X_processed)
        distortions = []

        for i, col in enumerate(feature_names):
            X_permuted = X_processed.copy()
            np.random.shuffle(X_permuted[:, i])
            permuted_embedding = reducer.transform(X_permuted)

            distortion = np.mean(np.linalg.norm(baseline_embedding - permuted_embedding, axis=1))
            distortions.append(distortion)

        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': distortions
        }).sort_values('Importance', ascending=False)

        plt.figure(figsize=plot_size)
        importance_df_sorted = importance_df.sort_values('Importance', ascending=True)

        colors = plt.cm.plasma(np.linspace(0.2, 1, len(importance_df_sorted)))
        bars = plt.barh(importance_df_sorted['Feature'],
                        importance_df_sorted['Importance'],
                        color=colors,
                        alpha=0.9)

        plt.xlabel('Distortion after Feature Permutation', fontsize=12)
        plt.title('UMAP Feature Importance (Embedding Sensitivity)', fontsize=14, pad=20)
        plt.grid(axis='x', linestyle='--', alpha=0.3)

        for bar in bars:
            width = bar.get_width()
            plt.text(width + 0.001,
                     bar.get_y() + bar.get_height() / 2,
                     f'{width:.4f}',
                     va='center',
                     fontsize=9)

        method_text = (
            f"Method: UMAP Compression\n"
            f"Components: {n_components}\n"
            f"Metric: {metric}\n"
            f"Scale Data: {scale_data}"
        )
        plt.annotate(method_text,
                     xy=(0.98, 0.02),
                     xycoords='axes fraction',
                     ha='right',
                     va='bottom',
                     fontsize=9,
                     bbox=dict(boxstyle='round', alpha=0.1))

        plt.tight_layout()
        plt.show()

    return reducer, compressed_df

-------

## 3.3.4 Autoencoder Feature Compression

<details>
<summary style="cursor: pointer">
<h2> { Understanding Autoencoders for Feature Compression } </h2>
</summary>
<h3> What are Autoencoders? </h3>
<p> Autoencoders are neural networks trained to reconstruct their input. The compressed hidden layer representation is a lower-dimensional encoding of the input data.</p>
<h3> It's role in Feature Importance / Compression: </h3>
<ul>
    <li> Learns compressed latent representations of high-dimensional input data.</li>
    <li> Can capture non-linear relationships and interactions between features.</li>
    <li> Often used for denoising, anomaly detection, and representation learning.</li>
</ul>
<h3> Resources: </h3>
<ol>
    <li><a href="https://www.youtube.com/watch?v=9zKuYvjFFS8" target="_blank">Autoencoders Explained (StatQuest)</a></li>
</ol>
</details>

##### Parameters:

- X: Features (DataFrame)
- model: A fitted model (optional, for aligning compressed features with model performance)
- encoding_dim: Size of the compressed (latent) space
- hidden_layers: List defining hidden layers size (optional, for deeper architectures)
- epochs: Number of training epochs
- batch_size: Batch size during training
- learning_rate: Learning rate for the optimizer
- scale_data: Whether to standardize features before training
- random_state: Random seed for reproducibility
- show_plot: Whether to plot feature contributions to latent space (optional)
- plot_size: Tuple indicating plot size (width, height)

##### Returns:

- Encoder model (to transform features)
- DataFrame of compressed feature representations
- Displays a plot of feature contributions (if show_plot=True)

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

def autoencoder_feature_compression(X,
                                     model=None,
                                     encoding_dim=10,
                                     hidden_layers=None,
                                     epochs=50,
                                     batch_size=32,
                                     learning_rate=0.001,
                                     scale_data=True,
                                     random_state=42,
                                     show_plot=True,
                                     plot_size=(12, 8)):

    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    tf.random.set_seed(random_state)
    np.random.seed(random_state)

    feature_names = X.columns.tolist()
    X_processed = X.copy()

    if scale_data:
        scaler = StandardScaler()
        X_processed = scaler.fit_transform(X_processed)
    else:
        X_processed = X_processed.values

    input_dim = X_processed.shape[1]

    # Build Autoencoder
    input_layer = Input(shape=(input_dim,))

    # Encoder
    x = input_layer
    if hidden_layers:
        for units in hidden_layers:
            x = Dense(units, activation='relu')(x)
    encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(1e-5))(x)

    # Decoder
    x = encoded
    if hidden_layers:
        for units in reversed(hidden_layers):
            x = Dense(units, activation='relu')(x)
    decoded = Dense(input_dim, activation='linear')(x)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    encoder = Model(inputs=input_layer, outputs=encoded)

    optimizer = Adam(learning_rate=learning_rate)
    autoencoder.compile(optimizer=optimizer, loss='mse')

    # Train Autoencoder
    autoencoder.fit(X_processed, X_processed,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    verbose=0)

    compressed_features = encoder.predict(X_processed)
    compressed_df = pd.DataFrame(compressed_features, columns=[f'Latent_{i+1}' for i in range(encoding_dim)])

    if show_plot:
        feature_importance = np.abs(encoder.get_weights()[0])  # Encoder's first layer weights
        importance_scores = np.sum(feature_importance, axis=1)

        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importance_scores
        }).sort_values('Importance', ascending=False)

        plt.figure(figsize=plot_size)
        importance_df_sorted = importance_df.sort_values('Importance', ascending=True)

        colors = plt.cm.viridis(np.linspace(0.2, 1, len(importance_df_sorted)))
        bars = plt.barh(importance_df_sorted['Feature'],
                        importance_df_sorted['Importance'],
                        color=colors,
                        alpha=0.9)

        plt.xlabel('Contribution to Latent Space', fontsize=12)
        plt.title('Autoencoder Feature Importance (Latent Contributions)', fontsize=14, pad=20)
        plt.grid(axis='x', linestyle='--', alpha=0.3)

        for bar in bars:
            width = bar.get_width()
            plt.text(width + 0.0005,
                     bar.get_y() + bar.get_height() / 2,
                     f'{width:.4f}',
                     va='center',
                     fontsize=9)

        method_text = (
            f"Method: Autoencoder Compression\n"
            f"Encoding Dim: {encoding_dim}\n"
            f"Scale Data: {scale_data}"
        )
        plt.annotate(method_text,
                     xy=(0.98, 0.02),
                     xycoords='axes fraction',
                     ha='right',
                     va='bottom',
                     fontsize=9,
                     bbox=dict(boxstyle='round', alpha=0.1))

        plt.tight_layout()
        plt.show()

    return encoder, compressed_df

---------

## 3.3.5 Factor Analysis

<details>
<summary style="cursor: pointer">
<h2> { Understanding Factor Analysis for Feature Extraction } </h2>
</summary>
<h3> What is Factor Analysis? </h3>
<p> Factor Analysis is a statistical method used to describe variability among observed, correlated variables in terms of fewer unobserved variables called factors.</p>
<h3> It's role in Feature Importance / Dimensionality Reduction: </h3>
<ul>
    <li> Assumes that the observed features are influenced by a smaller number of latent variables (factors).</li>
    <li> Especially useful in psychology, social sciences, and market research.</li>
    <li> Focuses more on modeling the structure rather than just variance like PCA.</li>
</ul>
<h3> Resources: </h3>
<ol>
    <li><a href="https://www.youtube.com/watch?v=Jkf-pGDdy7k" target="_blank">Factor Analysis Explained</a></li>
</ol>
</details>

##### Parameters:
    
- n_factors: Number of factors to extract (default=5, or determined automatically if kaiser_criterion=True)
- rotation: Type of rotation method ('varimax', 'quartimax', 'promax', 'oblimin', or None)
- max_iter: Maximum number of iterations for FA algorithm convergence
- scale_data: Whether to standardize features before factor analysis
- min_correlation: Minimum correlation threshold for displaying factor loadings
- kaiser_criterion: If True, automatically determine n_factors based on eigenvalues > 1
- variance_explained_threshold: Minimum cumulative variance explained threshold for factor selection
    
##### Returns:
    
- factor_model: Fitted factor analysis model
- factor_scores: DataFrame of factor scores for each observation
- feature_importance: DataFrame of feature importance scores

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
from scipy.stats import pearsonr

def factor_analysis_feature_importance(X,
                                      n_factors=5,
                                      rotation='varimax',
                                      max_iter=1000,
                                      scale_data=True,
                                      min_correlation=0.3,
                                      random_state=42,
                                      show_plot=True,
                                      plot_size=(12, 8),
                                      feature_contribution_method='loadings_squared',
                                      kaiser_criterion=True,
                                      variance_explained_threshold=0.8):
    
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)
    
    np.random.seed(random_state)
    
    feature_names = X.columns.tolist()
    X_processed = X.copy()
    
    # Standardize data if requested
    if scale_data:
        scaler = StandardScaler()
        X_processed = scaler.fit_transform(X_processed)
    else:
        X_processed = X_processed.values
    
    # Determine optimal number of factors if kaiser_criterion is True
    if kaiser_criterion:
        # Calculate correlation matrix and its eigenvalues
        corr_matrix = np.corrcoef(X_processed, rowvar=False)
        eigenvalues, _ = np.linalg.eig(corr_matrix)
        eigenvalues = np.real(eigenvalues)  # Ensure real values
        
        # Apply Kaiser criterion (eigenvalues > 1)
        k = sum(eigenvalues > 1)
        
        # Ensure we have at least 1 factor and at most n_features factors
        n_factors = max(min(k, X_processed.shape[1]), 1)
        
        # Further adjust n_factors based on variance explained threshold
        explained_variance_ratio = eigenvalues / sum(eigenvalues)
        cumulative_variance = np.cumsum(explained_variance_ratio)
        for i, var in enumerate(cumulative_variance):
            if var >= variance_explained_threshold:
                n_factors = min(n_factors, i + 1)
                break
    
    # Create and fit factor analysis model
    factor_model = FactorAnalysis(n_components=n_factors,
                                 rotation=rotation,
                                 max_iter=max_iter,
                                 random_state=random_state)
    
    factor_model.fit(X_processed)
    
    # Get factor loadings
    loadings = factor_model.components_.T
    
    # Calculate factor scores
    factor_scores = factor_model.transform(X_processed)
    factor_scores_df = pd.DataFrame(factor_scores, 
                                   columns=[f'Factor_{i+1}' for i in range(n_factors)])
    
    # Calculate variance explained by each factor
    # Since FactorAnalysis doesn't provide explained_variance directly,
    # we'll compute it from the loadings
    communalities = np.sum(loadings**2, axis=1)
    total_variance_explained = np.sum(communalities) / X_processed.shape[1]
    factor_variance = np.sum(loadings**2, axis=0) / X_processed.shape[1]
    variance_explained = factor_variance / sum(factor_variance) * total_variance_explained
    
    # Calculate feature importance based on the chosen method
    if feature_contribution_method == 'loadings_squared':
        # Sum of squared loadings across factors
        importance_scores = np.sum(loadings**2, axis=1)
    elif feature_contribution_method == 'communality':
        # Communality (proportion of each variable's variance explained by the factors)
        importance_scores = communalities
    elif feature_contribution_method == 'variance_weighted':
        # Weight loadings by variance explained by each factor
        weighted_loadings = loadings**2 * variance_explained
        importance_scores = np.sum(weighted_loadings, axis=1)
    else:
        # Default to squared loadings
        importance_scores = np.sum(loadings**2, axis=1)
    
    # Create feature importance DataFrame
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance_scores
    }).sort_values('Importance', ascending=False)
    
    # Create factor loadings DataFrame for interpretation
    loadings_df = pd.DataFrame(loadings, 
                             index=feature_names,
                             columns=[f'Factor_{i+1}' for i in range(n_factors)])
    
    if show_plot:
        plt.figure(figsize=plot_size)
        importance_df_sorted = importance_df.sort_values('Importance', ascending=True)
        
        colors = plt.cm.viridis(np.linspace(0.2, 1, len(importance_df_sorted)))
        bars = plt.barh(importance_df_sorted['Feature'],
                      importance_df_sorted['Importance'],
                      color=colors,
                      alpha=0.9)
        
        plt.xlabel('Feature Importance Score', fontsize=12)
        plt.title('Factor Analysis Feature Importance', fontsize=14, pad=20)
        plt.grid(axis='x', linestyle='--', alpha=0.3)
        
        for bar in bars:
            width = bar.get_width()
            plt.text(width + 0.005 * max(importance_scores),
                   bar.get_y() + bar.get_height() / 2,
                   f'{width:.4f}',
                   va='center',
                   fontsize=9)
        
        method_text = (
            f"Method: Factor Analysis\n"
            f"# Factors: {n_factors}\n"
            f"Rotation: {rotation}\n"
            f"Importance: {feature_contribution_method}\n"
            f"Total Var Explained: {total_variance_explained:.2f}"
        )
        plt.annotate(method_text,
                   xy=(0.98, 0.02),
                   xycoords='axes fraction',
                   ha='right',
                   va='bottom',
                   fontsize=9,
                   bbox=dict(boxstyle='round', alpha=0.1))
        
        plt.tight_layout()
        plt.show()
        
        # Additional plot: Factor loading heatmap
        plt.figure(figsize=plot_size)
        
        # Mask small loadings for clarity
        masked_loadings = loadings.copy()
        abs_loadings = np.abs(masked_loadings)
        mask = abs_loadings < min_correlation
        masked_loadings[mask] = 0
        
        # Plot heatmap
        im = plt.imshow(masked_loadings, aspect='auto', cmap='coolwarm', vmin=-1, vmax=1)
        plt.colorbar(im, label='Factor Loading')
        
        plt.xlabel('Factors', fontsize=12)
        plt.ylabel('Features', fontsize=12)
        plt.title('Factor Loadings Heatmap', fontsize=14)
        
        plt.xticks(np.arange(n_factors), [f'Factor {i+1}\n({variance_explained[i]:.2f})' for i in range(n_factors)])
        plt.yticks(np.arange(len(feature_names)), feature_names)
        
        # Annotate loadings
        for i in range(len(feature_names)):
            for j in range(n_factors):
                if abs(loadings[i, j]) >= min_correlation:
                    plt.text(j, i, f'{loadings[i, j]:.2f}',
                           ha='center', va='center',
                           color='white' if abs(loadings[i, j]) > 0.6 else 'black',
                           fontsize=8)
        
        plt.tight_layout()
        plt.show()
    
    # Create a dictionary to return multiple results
    return factor_model, factor_scores_df, importance_df, loadings_df


def interpret_factors(loadings_df, min_loading=0.5, max_features=3):
   
    factor_interpretations = {}
    
    for col in loadings_df.columns:
        # Get absolute loadings and sort
        abs_loadings = loadings_df[col].abs()
        sorted_loadings = abs_loadings.sort_values(ascending=False)
        
        # Get top features that meet minimum loading threshold
        top_features = sorted_loadings[sorted_loadings >= min_loading].index.tolist()[:max_features]
        
        # Create interpretation
        if top_features:
            feature_signs = ['+' if loadings_df.loc[feature, col] > 0 else '-' for feature in top_features]
            feature_desc = [f"{sign}{feature}" for feature, sign in zip(top_features, feature_signs)]
            factor_interpretations[col] = ", ".join(feature_desc)
        else:
            factor_interpretations[col] = "No strong loadings"
    
    return factor_interpretations
    
    print("\nFeature Importance:")
    print(importance)